In [123]:
import requests

from ape import networks, Contract
import pandas as pd
from pycoingecko import CoinGeckoAPI

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [125]:
badger_price = CoinGeckoAPI().get_price(["badger-dao"], "usd")['badger-dao']['usd']
badger_price

3.74

In [126]:
# endpoint and query for aura proposals
SNAPSHOT_SUBGRAPH = "https://hub.snapshot.org/graphql?"
PROPOSAL_INFO_QUERY = """
        query($proposal_id: String) {
          proposal(id: $proposal_id) {
            scores_total
            snapshot
            choices
            scores
          }
        }
        """
proposal_id_last = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"
proposal_id_current = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"

# endpoint for llama for cost per vote last round
LLAMA_DASHBOARD_URL = "https://api.llama.airforce//dashboard"
aura_dash = requests.post(LLAMA_DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
cost_per_vote_last_round = aura_dash['dashboard']['epochs'][-1]['dollarPerVlAsset']
cost_per_vote_after_fee = cost_per_vote_last_round * (1.04)

# check active voter in past proposal as ref
response_proposal = requests.post(
        SNAPSHOT_SUBGRAPH,
        json={"query": PROPOSAL_INFO_QUERY, "variables": {"proposal_id": proposal_id_last}},
    ).json()["data"]["proposal"]
response_proposal = requests.post(
        SNAPSHOT_SUBGRAPH,
        json={"query": PROPOSAL_INFO_QUERY, "variables": {"proposal_id": proposal_id_current}},
    ).json()["data"]["proposal"]

block_current_proposal = int(response_proposal["snapshot"])
aura_voting_actively_last_round = response_proposal["scores_total"]
data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', "${:.4f}".format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,15635887,6830592,$0.0660


In [127]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
strat_votes = vlAURA.getVotes('0x14f83ff95d4ec5e8812ddf42da1232b0ba1015e6', block_identifier=block_current_proposal) /1e18
council_fee = strat_votes*.1
treasury_votes = vlAURA.getVotes('0xA9ed98B5Fb8428d68664f3C5027c62A10d45826b', block_identifier=block_current_proposal) /1e18
total_vp_badgerdao = treasury_votes + council_fee
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
badger_aura_control = "{0:.3%}".format(pct_controlled_aura_by_badger)
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
badger_aura_control_active = "{0:.3%}".format(pct_controlled_aura_by_badger_active)
data = [[treasury_votes, council_fee, total_vp_badgerdao, badger_aura_control, badger_aura_control_active]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,182298.367156,91833.486207,274131.853364,3.396%,4.013%


In [128]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_aura_vebal_display = "{0:.3%}".format(pct_aura_vebal_controlled)
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_display = "{0:.3%}".format(pct_badger_vebal_controlled)
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_badger_vebal_acive_display = "{0:.3%}".format(pct_badger_vebal_controlled_active)
data = [[pct_aura_vebal_display, pct_badger_vebal_display, pct_badger_vebal_acive_display]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,25.450%,0.864%,1.021%


In [129]:
# caps & budger may vary along the quarters modify accordingly
GAUGE_CAP = .02
MAX_BRIBE = 16_000

vlAURA_req = total_vp_badgerdao/(pct_badger_vebal_controlled_active/GAUGE_CAP)
cost_of_max_cap = vlAURA_req * cost_per_vote_after_fee
badger_to_bribe_balancer = cost_of_max_cap / badger_price
bribe_reclycing = badger_to_bribe_balancer * (total_vp_badgerdao/vlAURA_req)
data = [[vlAURA_req, "${:.4f}".format(cost_of_max_cap), badger_to_bribe_balancer, bribe_reclycing]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes = pd.DataFrame(data, columns=headers)
df_bribes

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,536779.36656,$35436.8277,9475.087629,4838.903085


In [130]:
# amount of bribe based on allowance free for bribing elsewhere
bribe_overflow = MAX_BRIBE - badger_to_bribe_balancer
data = [[bribe_overflow]]
headers = ["Badger Bribe Overflow"]
df_bribes_overflow = pd.DataFrame(data, columns=headers)
df_bribes_overflow


,Badger Bribe Overflow
0,6524.912371
